In [1]:
import pandas as pd
import json

In [2]:
#import leavetimes data. due to size, it has to be split into chunks when reading in
leavetimes = pd.read_csv('rt_leavetimes_DB_2018.txt', delimiter=';', chunksize=10**6)

In [3]:
#import trips data
trips = pd.read_csv('rt_trips_DB_2018.txt',sep=";")

In [4]:
#map trip IDs to its lines and routes so that trip_route[trip_id]=[line,route]
trip_route={trip_id:route_id.split("_") for trip_id, route_id in zip(trips['TRIPID'], trips['ROUTEID'])}

In [5]:
"""iterate over leavetimes data set to create a dictionary that holds the information of which stations
were used how many times at which stage by which route of which line, i.e.:
lines[line][route][progress][stop_id]=number"""
lines={}
for chunk in leavetimes:
    for trip_id, progress, stop_id in zip(chunk["TRIPID"],chunk["PROGRNUMBER"],chunk["STOPPOINTID"]):
        routes=trip_route[trip_id]
        if routes[0] not in lines:
            lines[routes[0]]={routes[1]:{progress:{stop_id:1}}}
        elif routes[1] not in lines[routes[0]]:
            lines[routes[0]][routes[1]]={progress:{stop_id:1}}
        elif progress not in lines[routes[0]][routes[1]]:
            lines[routes[0]][routes[1]][progress]={stop_id:1}
        elif stop_id not in lines[routes[0]][routes[1]][progress]:
            lines[routes[0]][routes[1]][progress][stop_id]=1
        else:
            lines[routes[0]][routes[1]][progress][stop_id]+=1

In [6]:
with open('route_constitution.json', 'w') as out:
    json.dump(lines, out)